# Прогрессивный анализ влияния признаков на качество прогнозирования

Этот ноутбук предназначен для анализа того, как поэтапное добавление различных типов признаков влияет на качество прогнозирования временных рядов c помощью LLM.

## Этапы анализа:
1. **Базовая модель** - обучение только на ряду `close` (univariate)
2. **+ Аномалии** - добавление колонки аномалий
3. **+ Новости** - добавление взвешенной оценки новостей  
4. **+ Паттерны свечей** - добавление паттернов японских свечей
5. **+ Технические индикаторы** - добавление технических индикаторов
6. **+ TSFresh признаки** - добавление статистических свойств временных рядов
7. **+ PCA компоненты** - добавление сжатых признаков

## Метрики оценки:
- **RMSE** - среднеквадратичная ошибка
- **MAPE** - средняя абсолютная процентная ошибка  
- **DA** - точность направления (Directional Accuracy)

## Модель:
- **ChatGPT-4o-mini** из OPENROUTER 
- **Горизонт прогноза**: последние 20 дней


In [ ]:
import subprocess
import os
import sys

# Путь к pip в активном ядре
pip_path = os.path.join(sys.prefix, "bin", "pip")

subprocess.check_call([pip_path, "install", "requests", "langchain", "langchain-gigachat", "matplotlib", "scikit-learn", "pandas", "numpy", "seaborn", "gigachat", "openai"])

In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from langchain.schema import HumanMessage, SystemMessage
from langchain_gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage

import requests
import re
import json

# Библиотеки для анализа
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import os
from pathlib import Path

# Настройка отображения
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Библиотеки загружены успешно")

Библиотеки загружены успешно


In [7]:
# Конфигурация API ключей
API_KEYS = {
    'openrouter': "",
}

In [8]:
# Пути к данным
INPUT_PATH = '../../data/multivariate_series/'
OUTPUT_PATH = './progressive_analysis/'

# Создаем выходную папку если её нет
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Список тикеров
tickers = ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']

# Параметры анализа
FORECAST_HORIZON = 10  # Количество дней для прогноза
TEST_SIZE = 11         # Размер тестовой выборки (больше чем горизонт прогноза)

print(f"Параметры анализа:")
print(f"- Входные данные: {INPUT_PATH}")
print(f"- Результаты: {OUTPUT_PATH}")
print(f"- Горизонт прогноза: {FORECAST_HORIZON} дней")
print(f"- Размер тестовой выборки: {TEST_SIZE} дней")
print(f"- Тикеры для анализа: {tickers}")

# Загружаем данные
data = {}
for ticker in tickers:
    try:
        file_path = f"{INPUT_PATH}{ticker}_multivariate.csv"
        # 1) Читаем CSV, парсим timestamp
        df = pd.read_csv(file_path, parse_dates=['timestamp'])
        # 2) Убираем timezone (делаем tz-naive)
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_localize(None)
        # 3) Индексируем и сортируем
        df = df.set_index('timestamp').sort_index()
        # 4) Рейнжируем по реальным бизнес-дням
        bd_index = pd.date_range(df.index.min(), df.index.max(), freq='B')
        df = df.reindex(bd_index)
        # 5) Заполняем пропуски for all columns
        df = df.ffill().bfill()
        # 6) Переименовываем индекс
        df.index.name = 'timestamp'
        
        data[ticker] = df
        print(f"Загружен {ticker}: {df.shape}")
        
    except Exception as e:
        print(f"Ошибка загрузки {ticker}: {e}")

print(f"\nУспешно загружено {len(data)} тикеров")

Параметры анализа:
- Входные данные: ../../data/multivariate_series/
- Результаты: ./progressive_analysis/
- Горизонт прогноза: 10 дней
- Размер тестовой выборки: 11 дней
- Тикеры для анализа: ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']
Загружен AFLT: (2415, 918)
Загружен LKOH: (2415, 915)
Загружен MOEX: (2415, 918)
Загружен NVTK: (2415, 916)
Загружен PIKK: (2415, 911)
Загружен SBER: (2415, 909)
Загружен VKCO: (1241, 910)
Загружен VTBR: (1764, 905)
Загружен X5: (1547, 913)
Загружен YDEX: (2415, 918)

Успешно загружено 10 тикеров


## Вспомогательные функции


In [9]:
def calculate_directional_accuracy(actual, predicted):
    """
    Вычисляет точность направления (Directional Accuracy)
    
    Args:
        actual: фактические значения
        predicted: прогнозируемые значения
    
    Returns:
        DA: точность направления (от 0 до 1)
    """
    if len(actual) <= 1 or len(predicted) <= 1:
        return np.nan
    
    # Направление изменения фактических значений
    actual_direction = np.diff(actual) > 0
    
    # Направление изменения прогнозов
    predicted_direction = np.diff(predicted) > 0
    
    # Точность направления
    da = np.mean(actual_direction == predicted_direction)
    
    return da

def prepare_features_for_stage(df, stage):
    """
    Подготавливает признаки для определенного этапа анализа
    
    Args:
        df: DataFrame с данными
        stage: номер этапа (1-7)
    
    Returns:
        feature_columns: список колонок для использования
    """
    
    # Базовые колонки (всегда исключаем)
    base_exclude = ['date', 'daily_headlines', 'return']
    
    if stage == 1:
        # Этап 1: только close (univariate)
        return ['close']
    
    elif stage == 2:
        # Этап 2: + аномалии
        features = ['close', 'anomaly']
        return features
    
    elif stage == 3:
        # Этап 3: + новости оценкой
        features = ['close', 'anomaly', 'weighted_score_with_decay']
        return features

    elif stage == 4:
        # Этап 4: + новости текстом
        features = ['close', 'anomaly', 'daily_headlines']
        return features
    
    
    elif stage == 5:
        # Этап 5: + паттерны свечей
        features = ['close', 'anomaly', 'weighted_score_with_decay']

        # Добавляем свечи
        candels_cols = [col for col in df.columns if col in ['open', 'high', 'low', 'volume']]
        features.extend(candels_cols)

        return features
    
    elif stage == 6:
        # Этап 6: + технические индикаторы
        features = ['close', 'anomaly', 'weighted_score_with_decay']

        # Добавляем свечи
        candels_cols = [col for col in df.columns if col in ['open', 'high', 'low', 'volume']]
        features.extend(candels_cols)
        
        # Добавляем технические индикаторы
        tech_indicators = ['return', 'SMA_14', 'SMA_50', 'EMA_14', 'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 
                          'BB_hband', 'BB_lband', 'ATR_14', 'OBV', 'VWAP']
        tech_cols = [col for col in df.columns if any(indicator in col for indicator in tech_indicators)]
        features.extend(tech_cols)
        return features
    
    elif stage == 7:
        # Этап 7: + TSFresh признаки
        features = ['close', 'anomaly', 'weighted_score_with_decay']

        # Добавляем свечи
        candels_cols = [col for col in df.columns if col in ['open', 'high', 'low', 'volume']]
        features.extend(candels_cols)
        
        tech_indicators = ['returb', 'SMA_14', 'SMA_50', 'EMA_14', 'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 
                          'BB_hband', 'BB_lband', 'ATR_14', 'OBV', 'VWAP']
        tech_cols = [col for col in df.columns if any(indicator in col for indicator in tech_indicators)]
        features.extend(tech_cols)

        # Добавляем TSFresh признаки
        tsfresh_cols = ['value__cwt_coefficients__coeff_14__w10_withs_(2, 5, 10, 20)', 'value__minimun', 'value__maximin', 'value__mean']
        features.extend(tsfresh_cols)
        return features

    elif stage == 8:
        # Этап 8: + Картинка
        features = ['close', 'anomaly', 'weighted_score_with_decay']

        # Добавляем свечи
        candels_cols = [col for col in df.columns if col in ['open', 'high', 'low', 'volume']]
        features.extend(candels_cols)
        
        tech_indicators = ['return', 'SMA_14', 'SMA_50', 'EMA_14', 'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 
                          'BB_hband', 'BB_lband', 'ATR_14', 'OBV', 'VWAP']
        tech_cols = [col for col in df.columns if any(indicator in col for indicator in tech_indicators)]
        features.extend(tech_cols)

        # Добавляем TSFresh признаки
        tsfresh_cols = [col for col in df.columns if 'value__' in col]
        features.extend(tsfresh_cols)

        #TODO: Картинка в base64 (график + отображение тех.индикаторов, как для тредера)
        return features
    
    else:
        raise ValueError(f"Неподдерживаемый этап: {stage}")



In [14]:
def compute_patch_stats(patch):
    """Вычисление статистик для патча временного ряда"""
    ex = float(np.mean(patch))
    dx = float(np.std(patch))
    try:
        mode_val = float(statistics.mode([round(x, 2) for x in patch]))
    except statistics.StatisticsError:
        mode_val = None
    pct = (patch[-1] - patch[0]) / patch[0] * 100 if patch[0] != 0 else 0
    return ex, dx, mode_val, pct

def make_patch_messages(window_data, stage, patch_size=5):
    """
    Разбить временное окно на неперекрывающиеся патчи patch_size
    и вернуть list system-сообщений по каждому патчу.
    """
    msgs = []
    
    # Получаем признаки для данного этапа
    features = prepare_features_for_stage(window_data, stage)
    available_features = [f for f in features if f in window_data.columns]
    
    n_patches = len(window_data) // patch_size
    if n_patches == 0:
        n_patches = 1
        patch_size = len(window_data)
    
    for i in range(n_patches):
        start_idx = i * patch_size
        end_idx = min((i + 1) * patch_size, len(window_data))
        patch_data = window_data.iloc[start_idx:end_idx]
        
        # Формируем характеристики патча для разных этапов
        characteristics = []
        
        # Всегда добавляем цены
        if 'close' in available_features:
            prices = [f"{val:.2f}" for val in patch_data['close'].values]
            characteristics.append(f"Prices: {', '.join(prices)}")
        
        if stage >= 2 and 'anomaly' in available_features:
            # Этап 2+: добавляем аномалии
            anomalies = [str(int(val)) if not pd.isna(val) else "0" for val in patch_data['anomaly'].values]
            characteristics.append(f"Anomaly (0-no, 1-yes): {', '.join(anomalies)}")
        
        if stage >= 3 and 'weighted_score_with_decay' in available_features:
            # Этап 3+: добавляем новостную оценку
            scores = [f"{val*100:.0f}" if not pd.isna(val) else "0" for val in patch_data['weighted_score_with_decay'].values]
            characteristics.append(f"News sentiment (-100 to +100): {', '.join(scores)}")
        
        if stage >= 4 and 'daily_headlines' in available_features:
            # Этап 4+: добавляем заголовки новостей (только последний в патче)
            headlines = patch_data['daily_headlines'].dropna()
            if len(headlines) > 0:
                last_headline = str(headlines.iloc[-1])[:100] + ("..." if len(str(headlines.iloc[-1])) > 100 else "")
                characteristics.append(f"Latest news: {last_headline}")
        
        if stage >= 5:
            # Этап 5+: добавляем OHLV данные
            if 'open' in available_features:
                opens = [f"{val:.2f}" for val in patch_data['open'].values]
                characteristics.append(f"Open prices: {', '.join(opens)}")
            
            if 'high' in available_features:
                highs = [f"{val:.2f}" for val in patch_data['high'].values]
                characteristics.append(f"High prices: {', '.join(highs)}")
            
            if 'low' in available_features:
                lows = [f"{val:.2f}" for val in patch_data['low'].values]
                characteristics.append(f"Low prices: {', '.join(lows)}")
            
            if 'volume' in available_features:
                volumes = [f"{val:.0f}" if not pd.isna(val) else "0" for val in patch_data['volume'].values]
                characteristics.append(f"Volumes: {', '.join(volumes)}")
        
        if stage >= 6:
            # Этап 6+: добавляем технические индикаторы (выборочно, чтобы не перегружать)
            if 'RSI_14' in available_features:
                rsi_vals = [f"{val:.1f}" if not pd.isna(val) else "50" for val in patch_data['RSI_14'].values]
                characteristics.append(f"RSI: {', '.join(rsi_vals)}")
            
            if 'MACD' in available_features:
                macd_vals = [f"{val:.3f}" if not pd.isna(val) else "0" for val in patch_data['MACD'].values]
                characteristics.append(f"MACD: {', '.join(macd_vals)}")
            
            if 'return' in available_features:
                returns = [f"{val*100:.1f}%" if not pd.isna(val) else "0%" for val in patch_data['return'].values]
                characteristics.append(f"Returns: {', '.join(returns)}")
        
        if stage >= 7:
            # Этап 7+: добавляем статистические признаки (выборочно)
            tsfresh_features = [col for col in available_features if 'value__' in col]
            if tsfresh_features:
                # Берем первый доступный TSFresh признак
                feature = tsfresh_features[0]
                vals = [f"{val:.3f}" if not pd.isna(val) else "0" for val in patch_data[feature].values]
                feature_name = feature.replace('value__', '').replace('_', ' ')[:20]
                characteristics.append(f"Stat feature: {', '.join(vals)}")
        
        # Собираем все характеристики в одно сообщение
        content = f"Patch {i+1}: " + " | ".join(characteristics)
        
        msgs.append({"role": "system", "content": content})
    
    return msgs

def parse_llm_response(response_text):
    """Извлечение числового значения из ответа LLM"""
    try:
        # Поиск первого числа в тексте
        match = re.search(r"[-+]?\d*\.?\d+", str(response_text).strip())
        if match:
            return float(match.group(0))
        else:
            return np.nan
    except (ValueError, AttributeError):
        return np.nan

print("Вспомогательные функции для работы с патчами определены!")

Вспомогательные функции для работы с патчами определены!


In [11]:
class OpenRouterLLMPredictor:
    """Класс для прогнозирования через OpenRouter API"""
    
    def __init__(self, api_key, model_name="openai/gpt-4o-mini", max_retries=3, drop_threshold=0.20):
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com/", 
            "X-Title": "Financial Time Series Forecasting"
        }
        self.max_retries = max_retries
        self.drop_threshold = drop_threshold
    
    def predict(self, window_data, stage, window_size=10):
        """Прогноз следующего значения на основе временного окна"""
        
        # Создаем промпт
        prompt = create_llm_prompt(window_data, stage, window_size)
        if prompt is None:
            return np.nan
        
        prev_price = window_data['close'].iloc[-1]
        attempt = 0
        
        while attempt < self.max_retries:
            attempt += 1
            
            try:
                # Формируем сообщения
                messages = [
                    {
                        "role": "system",
                        "content": (
                            "You are a financial time series forecaster. "
                            "When asked to predict, return exactly one numeric value "
                            "and nothing else—no explanations, no units, no commentary."
                        )
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
                
                # Если не первая попытка - добавляем контекст о проблеме
                if attempt > 1:
                    messages.insert(1, {
                        "role": "system",
                        "content": (
                            "Note: Previous prediction was unrealistic. "
                            "Please reconsider market trends and provide a more plausible prediction. "
                            "Return only the next price as a single number."
                        )
                    })
                
                # Отправляем запрос
                response = requests.post(
                    self.api_url,
                    headers=self.headers,
                    json={
                        "model": self.model_name,
                        "messages": messages,
                        "max_tokens": 50,
                        "temperature": 0.1
                    },
                    timeout=30
                )
                response.raise_for_status()
                
                # Парсим ответ
                response_data = response.json()
                content = response_data["choices"][0]["message"]["content"]
                pred = parse_llm_response(content)
                
                # Проверка на аномальные значения
                if np.isnan(pred):
                    print(f"    Попытка {attempt}: не удалось извлечь число из '{content}'")
                    if attempt >= self.max_retries:
                        return prev_price
                    time.sleep(1)
                    continue
                
                # Проверка на сильное занижение/завышение
                if pred < prev_price * (1 - self.drop_threshold) or pred > prev_price * (1 + self.drop_threshold):
                    print(f"    Попытка {attempt}: аномальный прогноз {pred:.2f} (предыдущая цена {prev_price:.2f})")
                    if attempt < self.max_retries:
                        time.sleep(1)
                        continue
                    else:
                        # Возвращаем ограниченное значение
                        if pred < prev_price * (1 - self.drop_threshold):
                            return prev_price * (1 - self.drop_threshold)
                        else:
                            return prev_price * (1 + self.drop_threshold)
                
                return pred
                
            except Exception as e:
                print(f"    Ошибка попытки {attempt}: {e}")
                if attempt >= self.max_retries:
                    return prev_price
                time.sleep(1)
        
        return prev_price

print("Класс OpenRouterLLMPredictor определен!")


Класс OpenRouterLLMPredictor определен!


In [12]:
def evaluate_model_for_ticker_llm(df, ticker, stage, predictor, window_size=20):
    """
    Оценивает LLM модель для одного тикера на определенном этапе
    Использует стратегию "точка за точкой" для честного прогнозирования
    
    Args:
        df: DataFrame с данными тикера
        ticker: название тикера
        stage: номер этапа (1-7)
        predictor: экземпляр OpenRouterLLMPredictor
        window_size: размер окна для обучения
    
    Returns:
        results: словарь с метриками
    """
    
    try:
        # Подготавливаем признаки для этапа
        feature_columns = prepare_features_for_stage(df, stage)
        
        # Проверяем наличие всех колонок
        available_features = [col for col in feature_columns if col in df.columns]
        
        if len(available_features) == 0:
            print(f"  - Нет доступных признаков для {ticker} на этапе {stage}")
            return None
        
        # Удаляем строки с NaN в выбранных признаках (кроме daily_headlines)
        numeric_features = [f for f in available_features if f != 'daily_headlines']
        df_clean = df[available_features].copy()
        
        # Заполняем NaN в числовых признаках
        for feature in numeric_features:
            if feature in df_clean.columns:
                df_clean[feature] = df_clean[feature].fillna(method='ffill').fillna(method='bfill')
        
        if len(df_clean) < TEST_SIZE + window_size:
            print(f"  - Недостаточно данных для {ticker} на этапе {stage}")
            return None
        
        # Прогнозирование точка за точкой
        predictions = []
        actual_values = []
        
        start_idx = len(df_clean) - TEST_SIZE
        
        for i in range(FORECAST_HORIZON):
            current_idx = start_idx + i
            
            # Формируем окно для обучения
            window_start = max(0, current_idx - window_size)
            window_data = df_clean.iloc[window_start:current_idx]
            
            if len(window_data) < 5:  # Минимальное окно
                break
            
            # Делаем прогноз
            pred = predictor.predict(window_data, stage, window_size=min(10, len(window_data)))
            
            # Получаем фактическое значение
            if current_idx < len(df_clean):
                actual = df_clean.iloc[current_idx]['close']
                
                predictions.append(pred)
                actual_values.append(actual)
                
                print(f"    День {i+1}: прогноз={pred:.2f}, факт={actual:.2f}, ошибка={abs(pred-actual)/actual*100:.1f}%")
            else:
                break
        
        if len(predictions) == 0:
            print(f"  - Пустые прогнозы для {ticker} на этапе {stage}")
            return None
        
        # Преобразуем в numpy массивы
        predicted_values = np.array(predictions)
        actual_values = np.array(actual_values)
        
        # Убираем NaN значения
        valid_mask = ~(np.isnan(predicted_values) | np.isnan(actual_values))
        predicted_values = predicted_values[valid_mask]
        actual_values = actual_values[valid_mask]
        
        if len(predicted_values) == 0:
            print(f"  - Нет валидных прогнозов для {ticker} на этапе {stage}")
            return None
        
        # Вычисляем метрики
        rmse_value = np.sqrt(mean_squared_error(actual_values, predicted_values))
        mape_value = mean_absolute_percentage_error(actual_values, predicted_values) * 100
        da_value = calculate_directional_accuracy(actual_values, predicted_values)
        
        results = {
            'ticker': ticker,
            'stage': stage,
            'rmse': rmse_value,
            'mape': mape_value,
            'da': da_value,
            'feature_count': len(available_features),
            'predictions_count': len(predicted_values)
        }
        
        print(f"  - {ticker}: RMSE={rmse_value:.4f}, MAPE={mape_value:.2f}%, DA={da_value:.3f}, Features={len(available_features)}, Predictions={len(predicted_values)}")
        
        return results
        
    except Exception as e:
        print(f"  - Ошибка для {ticker} на этапе {stage}: {str(e)}")
        return None

print("Функция evaluate_model_for_ticker_llm определена")


Функция evaluate_model_for_ticker_llm определена


In [15]:
# Тестируем патчи для одного тикера
def test_patch_example(ticker='SBER', stage=3, patch_size=5):
    """Показывает пример патчей для отладки"""
    if ticker not in data:
        print(f"Тикер {ticker} не найден")
        return
    
    df = data[ticker]
    feature_columns = prepare_features_for_stage(df, stage)
    
    # Проверяем наличие всех колонок
    available_features = [col for col in feature_columns if col in df.columns]
    print(f"📊 Тикер: {ticker}, Этап: {stage}")
    print(f"🔧 Доступные признаки: {available_features}")
    
    # Удаляем строки с NaN в выбранных признаках (кроме daily_headlines)
    numeric_features = [f for f in available_features if f != 'daily_headlines']
    df_clean = df[available_features].copy()
    
    # Заполняем NaN в числовых признаках
    for feature in numeric_features:
        if feature in df_clean.columns:
            df_clean[feature] = df_clean[feature].fillna(method='ffill').fillna(method='bfill')
    
    # Берем последние 15 дней для примера
    window_data = df_clean.tail(15)
    
    # Создаем патчи
    patches = make_patch_messages(window_data, stage, patch_size)
    
    print(f"\\n📝 Пример патчей (размер патча: {patch_size}):")
    print("=" * 80)
    for i, patch in enumerate(patches):
        print(f"Патч {i+1}:")
        print(patch['content'])
        print("-" * 40)
    print("=" * 80)
    
    return patches

# Тестируем разные этапы
print("🧪 ТЕСТИРОВАНИЕ ПАТЧЕЙ")
print("="*50)
test_patch_example('SBER', 1, 5)
print("\\n" + "-"*80 + "\\n")
test_patch_example('SBER', 3, 5)
print("\\n" + "-"*80 + "\\n")
test_patch_example('SBER', 6, 5)


🧪 ТЕСТИРОВАНИЕ ПАТЧЕЙ
📊 Тикер: SBER, Этап: 1
🔧 Доступные признаки: ['close']
\n📝 Пример патчей (размер патча: 5):
Патч 1:
Patch 1: Prices: 292.46, 299.81, 296.50, 297.99, 300.80
----------------------------------------
Патч 2:
Patch 2: Prices: 304.80, 300.01, 308.12, 312.25, 310.00
----------------------------------------
Патч 3:
Patch 3: Prices: 310.16, 316.69, 314.23, 309.41, 307.80
----------------------------------------
\n--------------------------------------------------------------------------------\n
📊 Тикер: SBER, Этап: 3
🔧 Доступные признаки: ['close', 'anomaly', 'weighted_score_with_decay']
\n📝 Пример патчей (размер патча: 5):
Патч 1:
Patch 1: Prices: 292.46, 299.81, 296.50, 297.99, 300.80 | Anomaly (0-no, 1-yes): 0, 0, 0, 0, 0 | News sentiment (-100 to +100): 0, 0, 0, 0, -39
----------------------------------------
Патч 2:
Patch 2: Prices: 304.80, 300.01, 308.12, 312.25, 310.00 | Anomaly (0-no, 1-yes): 0, 0, 0, 0, 0 | News sentiment (-100 to +100): -46, -72, 0, 71, 0
------

[{'role': 'system',
  'content': 'Patch 1: Prices: 292.46, 299.81, 296.50, 297.99, 300.80 | Anomaly (0-no, 1-yes): 0, 0, 0, 0, 0 | News sentiment (-100 to +100): 0, 0, 0, 0, -39 | Open prices: 294.09, 293.00, 301.49, 296.51, 298.00 | High prices: 296.75, 300.48, 302.40, 300.45, 303.30 | Low prices: 288.27, 292.80, 295.13, 295.60, 296.06 | Volumes: 6286284, 5041821, 3476895, 2425871, 3010350 | RSI: 42.1, 48.5, 45.7, 47.1, 49.8 | MACD: -6.616, -5.743, -3.902, -3.548, -3.007 | Returns: -0.3%, 2.5%, -1.7%, 0.5%, 0.9%'},
 {'role': 'system',
  'content': 'Patch 2: Prices: 304.80, 300.01, 308.12, 312.25, 310.00 | Anomaly (0-no, 1-yes): 0, 0, 0, 0, 0 | News sentiment (-100 to +100): -46, -72, 0, 71, 0 | Open prices: 300.89, 302.70, 302.67, 308.50, 313.00 | High prices: 305.07, 303.85, 308.70, 315.00, 314.98 | Low prices: 300.41, 296.25, 301.76, 306.55, 305.55 | Volumes: 3757980, 4614792, 5126013, 6448201, 5885119 | RSI: 53.4, 48.9, 55.7, 58.7, 56.5 | MACD: -2.229, -1.977, -1.109, -0.088, 0.534

In [27]:
# Тест одиночного прогноза
def test_single_prediction(ticker='SBER', stage=3, patch=3):
    """Тестирует одиночный прогноз для отладки"""
    if ticker not in data:
        print(f"Тикер {ticker} не найден")
        return
    
    # Создаем тестовый предиктор
    test_predictor = OpenRouterPredictor(
        api_key=API_KEYS['openrouter'],
        model_name="openai/gpt-4o-mini-2024-07-18",
        max_retries=1,
        drop_threshold=0.15
    )
    
    df = data[ticker]
    feature_columns = prepare_features_for_stage(df, stage)
    
    # Проверяем наличие всех колонок
    available_features = [col for col in feature_columns if col in df.columns]
    print(f"📊 Тестовый прогноз для {ticker}, Этап: {stage}")
    print(f"🔧 Доступные признаки: {available_features}")
    
    # Подготавливаем данные
    numeric_features = [f for f in available_features if f != 'daily_headlines']
    df_clean = df[available_features].copy()
    
    # Заполняем NaN в числовых признаках
    for feature in numeric_features:
        if feature in df_clean.columns:
            df_clean[feature] = df_clean[feature].fillna(method='ffill').fillna(method='bfill')
    
    # Берем окно для прогноза (последние 20 дней, исключая последний день)
    window_data = df_clean.iloc[-21:-1]  # 20 дней
    actual_price = df_clean.iloc[-1]['close']  # Фактическая цена последнего дня
    prev_price = window_data.iloc[-1]['close']  # Цена предыдущего дня
    
    print(f"💰 Предыдущая цена: {prev_price:.2f}")
    print(f"💰 Фактическая цена: {actual_price:.2f}")
    print(f"📈 Фактическое изменение: {((actual_price - prev_price) / prev_price * 100):+.2f}%")
    
    # Делаем прогноз
    print("\\n🤖 Отправляем запрос к LLM...")
    predicted_price = test_predictor.predict(window_data, stage, 1)
    
    if not np.isnan(predicted_price):
        error_pct = abs(predicted_price - actual_price) / actual_price * 100
        predicted_change = (predicted_price - prev_price) / prev_price * 100
        
        print(f"🎯 Прогнозируемая цена: {predicted_price:.2f}")
        print(f"📊 Прогнозируемое изменение: {predicted_change:+.2f}%") 
        print(f"❌ Абсолютная ошибка: {error_pct:.2f}%")
        
        # Проверка направления
        actual_direction = "рост" if actual_price > prev_price else "падение"
        predicted_direction = "рост" if predicted_price > prev_price else "падение"
        direction_correct = actual_direction == predicted_direction
        
        print(f"🧭 Направление: факт={actual_direction}, прогноз={predicted_direction} ({'✅' if direction_correct else '❌'})")
    else:
        print("❌ Не удалось получить прогноз")
    
    return predicted_price

# Запускаем тест (раскомментируйте для тестирования)
print("🧪 ТЕСТИРОВАНИЯ ОДИНОЧНОГО ПРОГНОЗА")
print("="*50)
test_single_prediction('SBER', 1)
print("\\n" + "-"*50 + "\\n")
test_single_prediction('SBER', 7)

🧪 ТЕСТИРОВАНИЯ ОДИНОЧНОГО ПРОГНОЗА
📊 Тестовый прогноз для SBER, Этап: 1
🔧 Доступные признаки: ['close']
💰 Предыдущая цена: 309.41
💰 Фактическая цена: 307.80
📈 Фактическое изменение: -0.52%
\n🤖 Отправляем запрос к LLM...
🎯 Прогнозируемая цена: 312.50
📊 Прогнозируемое изменение: +1.00%
❌ Абсолютная ошибка: 1.53%
🧭 Направление: факт=падение, прогноз=рост (❌)
\n--------------------------------------------------\n
📊 Тестовый прогноз для SBER, Этап: 7
🔧 Доступные признаки: ['close', 'anomaly', 'weighted_score_with_decay', 'open', 'high', 'low', 'volume', 'SMA_14', 'SMA_50', 'EMA_14', 'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_diff', 'BB_hband', 'BB_lband', 'ATR_14', 'OBV', 'VWAP', 'value__mean']
💰 Предыдущая цена: 309.41
💰 Фактическая цена: 307.80
📈 Фактическое изменение: -0.52%
\n🤖 Отправляем запрос к LLM...
🎯 Прогнозируемая цена: 311.50
📊 Прогнозируемое изменение: +0.68%
❌ Абсолютная ошибка: 1.20%
🧭 Направление: факт=падение, прогноз=рост (❌)


311.5

In [ ]:
def evaluate_llm_ticker(df, ticker, stage, predictor, window_size=20, patch_size=5):
    """
    Оценивает LLM модель для одного тикера на определенном этапе
    Использует стратегию "точка за точкой" с патчами
    """
    
    try:
        # Подготавливаем признаки для этапа
        feature_columns = prepare_features_for_stage(df, stage)
        
        # Проверяем наличие всех колонок
        available_features = [col for col in feature_columns if col in df.columns]
        
        if len(available_features) == 0:
            print(f"  - Нет доступных признаков для {ticker} на этапе {stage}")
            return None
        
        # Подготавливаем данные
        numeric_features = [f for f in available_features if f != 'daily_headlines']
        df_clean = df[available_features].copy()
        
        # Заполняем NaN в числовых признаках
        for feature in numeric_features:
            if feature in df_clean.columns:
                df_clean[feature] = df_clean[feature].fillna(method='ffill').fillna(method='bfill')
        
        if len(df_clean) < TEST_SIZE + window_size:
            print(f"  - Недостаточно данных для {ticker} на этапе {stage}")
            return None
        
        # Прогнозирование точка за точкой
        predictions = []
        actual_values = []
        
        start_idx = len(df_clean) - TEST_SIZE
        
        for i in range(FORECAST_HORIZON):
            current_idx = start_idx + i
            
            # Формируем окно для обучения
            window_start = max(0, current_idx - window_size)
            window_data = df_clean.iloc[window_start:current_idx]
            
            if len(window_data) < 5:  # Минимальное окно
                break
            
            # Делаем прогноз
            pred = predictor.predict(window_data, stage, patch_size=patch_size)
            
            # Получаем фактическое значение
            if current_idx < len(df_clean):
                actual = df_clean.iloc[current_idx]['close']
                
                predictions.append(pred)
                actual_values.append(actual)
                
                print(f"    День {i+1}: прогноз={pred:.2f}, факт={actual:.2f}, ошибка={abs(pred-actual)/actual*100:.1f}%")
            else:
                break
        
        if len(predictions) == 0:
            print(f"  - Пустые прогнозы для {ticker} на этапе {stage}")
            return None
        
        # Преобразуем в numpy массивы
        predicted_values = np.array(predictions)
        actual_values = np.array(actual_values)
        
        # Убираем NaN значения
        valid_mask = ~(np.isnan(predicted_values) | np.isnan(actual_values))
        predicted_values = predicted_values[valid_mask]
        actual_values = actual_values[valid_mask]
        
        if len(predicted_values) == 0:
            print(f"  - Нет валидных прогнозов для {ticker} на этапе {stage}")
            return None
        
        # Вычисляем метрики
        rmse_value = np.sqrt(mean_squared_error(actual_values, predicted_values))
        mape_value = mean_absolute_percentage_error(actual_values, predicted_values) * 100
        da_value = calculate_directional_accuracy(actual_values, predicted_values)
        
        results = {
            'ticker': ticker,
            'stage': stage,
            'rmse': rmse_value,
            'mape': mape_value,
            'da': da_value,
            'feature_count': len(available_features),
            'predictions_count': len(predicted_values)
        }
        
        print(f"  - {ticker}: RMSE={rmse_value:.4f}, MAPE={mape_value:.2f}%, DA={da_value:.3f}, Features={len(available_features)}")
        
        return results
        
    except Exception as e:
        print(f"  - Ошибка для {ticker} на этапе {stage}: {str(e)}")
        return None

print("Функция evaluate_llm_ticker определена")


In [ ]:
## 🚀 Инструкция по запуску прогрессивного анализа

### Подготовка:
1. **Убедитесь, что API ключ OpenRouter настроен** в ячейке с `API_KEYS`
2. **Проверьте, что данные загружены** - должны быть доступны multivariate файлы для всех тикеров
3. **Протестируйте промпты** - запустите ячейку с `test_prompt_example()` чтобы увидеть, как выглядят промпты для разных этапов

### Тестирование:
```python
# Протестируйте одиночный прогноз перед полным анализом
test_single_prediction('SBER', 1)  # Базовая модель
test_single_prediction('SBER', 3)  # С новостями
```

### Запуск полного анализа:
- Запустите ячейку с основным циклом анализа
- **Внимание**: Полный анализ может занять **2-3 часа** (7 этапов × 10 тикеров × 10 прогнозов × 2 секунды = ~2300 запросов к API)
- Для сокращения времени можно:
  - Уменьшить количество тикеров в списке `tickers`
  - Уменьшить `FORECAST_HORIZON` (сейчас 10 дней)
  - Увеличить паузу `time.sleep()` если получаете ошибки rate limit

### Этапы анализа:
1. **Этап 1**: Только цены закрытия (baseline)
2. **Этап 2**: + Индикаторы аномалий
3. **Этап 3**: + Новостная оценка (числовой скор)
4. **Этап 4**: + Текст новостей (заголовки)
5. **Этап 5**: + OHLV данные (свечи)
6. **Этап 6**: + Технические индикаторы (SMA, RSI, MACD и т.д.)
7. **Этап 7**: + TSFresh статистические признаки

### Результаты:
- Будут сохранены в папку `./progressive_analysis/`
- Графики изменения метрик по этапам
- CSV файлы с детальными результатами
- Анализ влияния каждого типа признаков на качество прогнозирования


In [ ]:
# Основной цикл прогрессивного анализа с патчами

# Определяем названия этапов
stage_names = {
    1: "Базовая модель (close)",
    2: "+ Аномалии", 
    3: "+ Новости (оценка)",
    4: "+ Новости (текст)",
    5: "+ Свечи (OHLV)",
    6: "+ Технические индикаторы",
    7: "+ TSFresh признаки"
}

# Контейнер для результатов
all_results = []
stage_summaries = []

print("🚀 Начинаем прогрессивный анализ влияния признаков с LLM (патчи)\\n")

# Проходим по всем этапам
for stage in range(1, 8):  # 7 этапов
    print(f"📊 ЭТАП {stage}: {stage_names[stage]}")
    print("=" * 50)
    
    # Создаем предиктор для этого этапа
    predictor = OpenRouterPredictor(
        api_key=API_KEYS['openrouter'],
        model_name="openai/gpt-4o-mini-2024-07-18",
        max_retries=2,
        drop_threshold=0.15
    )
    
    stage_results = []
    
    # Оцениваем каждый тикер на текущем этапе
    for ticker in tickers:
        if ticker in data:
            print(f"\\n  Обрабатываем {ticker}...")
            result = evaluate_llm_ticker(data[ticker], ticker, stage, predictor, window_size=20, patch_size=5)
            if result is not None:
                all_results.append(result)
                stage_results.append(result)
            
            # Небольшая пауза между запросами к API
            time.sleep(2)
    
    # Вычисляем средние метрики по этапу
    if stage_results:
        avg_rmse = np.mean([r['rmse'] for r in stage_results])
        avg_mape = np.mean([r['mape'] for r in stage_results])
        avg_da = np.mean([r['da'] for r in stage_results if not np.isnan(r['da'])])
        avg_features = np.mean([r['feature_count'] for r in stage_results])
        
        stage_summary = {
            'stage': stage,
            'stage_name': stage_names[stage],
            'avg_rmse': avg_rmse,
            'avg_mape': avg_mape,
            'avg_da': avg_da,
            'avg_features': avg_features,
            'ticker_count': len(stage_results)
        }
        
        stage_summaries.append(stage_summary)
        
        print(f"\\n📈 Средние результаты этапа {stage}:")
        print(f"   RMSE: {avg_rmse:.4f}")
        print(f"   MAPE: {avg_mape:.2f}%")
        print(f"   DA: {avg_da:.3f}")
        print(f"   Признаков: {avg_features:.1f}")
        print(f"   Успешных тикеров: {len(stage_results)}/{len(tickers)}")
    else:
        print(f"❌ Нет успешных результатов для этапа {stage}")
    
    print("\\n" + "="*50 + "\\n")

print(f"✅ Анализ завершен! Собрано {len(all_results)} результатов из {len(stage_summaries)} этапов")


In [ ]:
## 🚀 Как запустить прогрессивный анализ

### 1. Предварительное тестирование

**Сначала протестируйте патчи:**
```python
# Запустите ячейку с test_patch_example() чтобы посмотреть как выглядят патчи
```

**Затем протестируйте одиночный прогноз:**
```python
# Раскомментируйте и запустите тестирование в ячейке test_single_prediction()
test_single_prediction('SBER', 1)  # Базовая модель
test_single_prediction('SBER', 3)  # С новостями  
```

### 2. Запуск полного анализа

**Для полного анализа запустите ячейку с основным циклом**

**⚠️ Внимание:**
- Полный анализ займет **2-3 часа** 
- Будет выполнено ~1400 запросов к API (7 этапов × 10 тикеров × 10 прогнозов × 2 секунды)
- Стоимость: примерно $5-10 в зависимости от тарифов OpenRouter

**🔧 Для ускорения тестирования:**
```python
# Уменьшите количество тикеров:
tickers = ['SBER', 'MOEX', 'LKOH']  # Только 3 тикера

# Или уменьшите горизонт прогноза:
FORECAST_HORIZON = 5  # Вместо 10 дней

# Или протестируйте только несколько этапов:
for stage in range(1, 4):  # Только первые 3 этапа
```

### 3. Результаты

После завершения анализа:
- Результаты сохранятся в `./progressive_analysis/`
- Вы увидите как каждый тип признаков влияет на качество прогнозирования
- Графики покажут изменение RMSE, MAPE и DA по этапам


In [17]:
class OpenRouterPredictor:
    """Базовый класс для LLM моделей через OpenRouter API"""
    
    def __init__(self, api_key, model_name, max_retries=1, drop_threshold=0.20):
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com/", 
            "X-Title": "Financial Time Series Forecasting"
        }
        self.max_retries = max_retries
        self.drop_threshold = drop_threshold
    
    def predict(self, window_data, stage, patch_size=5):
        """Прогноз следующего значения на основе временного окна"""
        prev_price = window_data['close'].iloc[-1]
        attempt = 0
        
        while attempt < self.max_retries:
            attempt += 1
            
            try:
                # Формируем сообщения по патчам
                msgs = make_patch_messages(window_data, stage, patch_size)
                
                # Добавляем system prompt
                msgs.insert(0, {
                    "role": "system",
                    "content": (
                        "You are a financial time series forecaster. "
                        "When asked to predict, return exactly one numeric value "
                        "and nothing else—no explanations, no units, no commentary."
                    )
                })
                
                # Если не первая попытка - добавляем контекст о проблеме
                if attempt > 1:
                    msgs.insert(1, {
                        "role": "system",
                        "content": (
                            "Note: Previous prediction was unrealistic. "
                            "Please reconsider market trends and provide a more plausible prediction. "
                            "Return only the next price as a single number."
                        )
                    })
                
                # User prompt
                msgs.append({
                    "role": "user",
                    "content": "Based on these financial characteristics, predict the next closing price value. Output only the next closing price as a number."
                })
                
                # Отправляем запрос
                response = requests.post(
                    self.api_url,
                    headers=self.headers,
                    json={
                        "model": self.model_name,
                        "messages": msgs,
                        "max_tokens": 50,
                        "temperature": 0.1
                    },
                    timeout=30
                )
                response.raise_for_status()
                
                # Парсим ответ
                response_data = response.json()
                content = response_data["choices"][0]["message"]["content"]
                pred = parse_llm_response(content)
                
                # Проверка на аномальные значения
                if np.isnan(pred):
                    print(f"    Попытка {attempt}: не удалось извлечь число из '{content}'")
                    if attempt >= self.max_retries:
                        return prev_price
                    time.sleep(1)
                    continue
                
                # Проверка на сильное занижение/завышение
                if pred < prev_price * (1 - self.drop_threshold) or pred > prev_price * (1 + self.drop_threshold):
                    print(f"    Попытка {attempt}: аномальный прогноз {pred:.2f} (предыдущая цена {prev_price:.2f})")
                    if attempt < self.max_retries:
                        time.sleep(1)
                        continue
                    else:
                        # Возвращаем ограниченное значение
                        if pred < prev_price * (1 - self.drop_threshold):
                            return prev_price * (1 - self.drop_threshold)
                        else:
                            return prev_price * (1 + self.drop_threshold)
                
                return pred
                
            except Exception as e:
                print(f"    Ошибка попытки {attempt}: {e}")
                if attempt >= self.max_retries:
                    return prev_price
                time.sleep(1)
        
        return prev_price

print("Базовый класс OpenRouterPredictor определен!")

Базовый класс OpenRouterPredictor определен!


In [ ]:
def evaluate_model_for_ticker_llm():
    #TODO

In [ ]:
def evaluate_model_for_ticker(df, ticker, stage):
    """
    Обучает модель и оценивает её для одного тикера на определенном этапе
    Использует стратегию "точка за точкой" для честного прогнозирования
    
    Args:
        df: DataFrame с данными тикера
        ticker: название тикера
        stage: номер этапа (1-7)
    
    Returns:
        results: словарь с метриками и важностью признаков
    """
    
    #try:
    # Подготавливаем признаки для этапа
    feature_columns = prepare_features_for_stage(df, stage)
    
    # Проверяем наличие всех колонок
    available_features = [col for col in feature_columns if col in df.columns]
    
    if len(available_features) == 0:
        print(f"  - Нет доступных признаков для {ticker} на этапе {stage}")
        return None
    
    # Удаляем строки с NaN в выбранных признаках
    df_clean = df[available_features].dropna()
    
    if len(df_clean) < TEST_SIZE + 10:  # Минимум данных для обучения
        print(f"  - Недостаточно данных для {ticker} на этапе {stage}")
        return None
    
    # Готовим данные для DARTS
    if stage == 1:
        # Univariate модель по стратегии extending window
        ts = TimeSeries.from_dataframe(df_clean, time_col=None, value_cols='close')
        current_ts = ts[:-TEST_SIZE]   # начальное обучающее окно
        predictions = []

        for i in range(FORECAST_HORIZON):
            # 1) Переобучаем модель на всем current_ts
            model = RandomForest(lags=14, random_state=42)
            model.fit(current_ts)

            # 2) Делаем прогноз на 1 шаг вперед
            pred = model.predict(n=1)
            y_pred = pred.values().flatten()[0]
            predictions.append(y_pred)

            # 3) Добавляем фактическое значение в окно
            next_time = ts.time_index[len(current_ts)]
            y_true = ts.values()[len(current_ts)][0]
            s = pd.Series([y_true], index=[next_time])
            actual_ts = TimeSeries.from_series(
                s,
                fill_missing_dates=False,
                freq=ts.freq
            )
            current_ts = current_ts.append(actual_ts)

        feature_importance = {}
        
    else:
        # Multivariate модель
        target_col = 'close'
        past_covariates_cols = [col for col in available_features if col != target_col]
        
        if len(past_covariates_cols) == 0:
            # Fallback к univariate если нет ковариат
            ts = TimeSeries.from_dataframe(df_clean, time_col=None, value_cols=target_col)
 
            # Прогнозирование точка за точкой
            predictions = []
            current_ts = ts[:-TEST_SIZE]
            
            for i in range(FORECAST_HORIZON):
                # 1) Переобучаем модель на всем current_ts
                model = RandomForest(lags=14, random_state=42)
                model.fit(current_ts)
    
                # 2) Делаем прогноз на 1 шаг вперед
                pred = model.predict(n=1)
                y_pred = pred.values().flatten()[0]
                predictions.append(y_pred)
    
                # 3) Добавляем фактическое значение в окно
                next_time = ts.time_index[len(current_ts)]
                y_true = ts.values()[len(current_ts)][0]
                s = pd.Series([y_true], index=[next_time])
                actual_ts = TimeSeries.from_series(
                    s,
                    fill_missing_dates=False,
                    freq=ts.freq
                )
                current_ts = current_ts.append(actual_ts)
            
            feature_importance = {}
            
        else:
            # Создаем TimeSeries для цели и ковариат
            target_ts = TimeSeries.from_dataframe(df_clean, time_col=None, value_cols=target_col)
            past_covariates_ts = TimeSeries.from_dataframe(df_clean, time_col=None, value_cols=past_covariates_cols)

            # чтобы в конце можно было читать actual_values из ts
            ts = target_ts
            
            # Прогнозирование точка за точкой
            predictions = []
            current_target = target_ts[:-TEST_SIZE]
            current_covariates = past_covariates_ts[:-TEST_SIZE]
            
            for i in range(FORECAST_HORIZON):
                # 1) Переобучаем multivariate модель
                model = RandomForest(lags=14, lags_past_covariates=7, random_state=42)
                model.fit(series=current_target, past_covariates=current_covariates)
    
                # 2) Прогноз
                pred = model.predict(n=1, past_covariates=current_covariates)
                y_pred = pred.values().flatten()[0]
                predictions.append(y_pred)
    
                # 3) Добавляем фактическое значение таргета
                next_t = target_ts.time_index[len(current_target)]
                y_true = target_ts.values()[len(current_target)][0]
                s_y = pd.Series([y_true], index=[next_t])
                actual_y_ts = TimeSeries.from_series(
                    s_y, fill_missing_dates=False, freq=target_ts.freq
                )
                current_target = current_target.append(actual_y_ts)
    
                # 4) Добавляем фактические ковариаты
                next_t = past_covariates_ts.time_index[len(current_covariates)]
                x_true = past_covariates_ts.values()[len(current_covariates)].flatten()
                df_x = pd.DataFrame([x_true], index=[next_t], columns=past_covariates_cols)
                actual_x_ts = TimeSeries.from_dataframe(
                    df_x, time_col=None, value_cols=past_covariates_cols,
                    fill_missing_dates=False, freq=past_covariates_ts.freq
                )
                current_covariates = current_covariates.append(actual_x_ts)
            
            # Извлекаем важность признаков
            if hasattr(model.model, 'feature_importances_'):
                importance_values = model.model.feature_importances_
                # Создаем названия признаков (lags + past_covariates)
                feature_names = []
                for lag in range(1, 15):  # lags=14
                    feature_names.append(f'{target_col}_lag_{lag}')
                for lag in range(1, 8):   # lags_past_covariates=7
                    for col in past_covariates_cols:
                        feature_names.append(f'{col}_lag_{lag}')
                
                feature_importance = dict(zip(feature_names[:len(importance_values)], importance_values))
            else:
                feature_importance = {}
    
    # Получаем реальные значения для сравнения
    actual_values = ts[-TEST_SIZE:-TEST_SIZE+FORECAST_HORIZON].values().flatten()
    predicted_values = np.array(predictions)
    
    # Убеждаемся что размеры совпадают
    min_length = min(len(actual_values), len(predicted_values))
    actual_values = actual_values[:min_length]
    predicted_values = predicted_values[:min_length]
    
    if min_length == 0:
        print(f"  - Пустые прогнозы для {ticker} на этапе {stage}")
        return None
    
    # RMSE
    rmse_value = np.sqrt(mean_squared_error(actual_values, predicted_values))
    
    # MAPE
    mape_value = mean_absolute_percentage_error(actual_values, predicted_values) * 100
    
    # DA (Directional Accuracy)
    da_value = calculate_directional_accuracy(actual_values, predicted_values)
    
    results = {
        'ticker': ticker,
        'stage': stage,
        'rmse': rmse_value,
        'mape': mape_value,
        'da': da_value,
        'feature_count': len(available_features),
        'feature_importance': feature_importance
    }
    
    print(f"  - {ticker}: RMSE={rmse_value:.4f}, MAPE={mape_value:.2f}%, DA={da_value:.3f}, Features={len(available_features)}")
    
    return results
        
    # except Exception as e:
    #     print(f"  - Ошибка для {ticker} на этапе {stage}: {str(e)}")
    #     return None

print("Вспомогательные функции определены")

## Прогрессивный анализ по этапам


In [ ]:
# Создаем предиктор
predictor = OpenRouterLLMPredictor(
    api_key=API_KEYS['openrouter'],
    model_name="openai/gpt-4o-mini-2024-07-18",
    max_retries=2,
    drop_threshold=0.15
)

# Определяем названия этапов
stage_names = {
    1: "Базовая модель (close)",
    2: "+ Аномалии", 
    3: "+ Новости (оценка)",
    4: "+ Новости (текст)",
    5: "+ Свечи (OHLV)",
    6: "+ Технические индикаторы",
    7: "+ TSFresh признаки"
}

# Контейнер для результатов
all_results = []
stage_summaries = []

print("🚀 Начинаем прогрессивный анализ влияния признаков LLM\\n")

# Проходим по всем этапам
for stage in range(1, 8):  # 7 этапов
    print(f"📊 ЭТАП {stage}: {stage_names[stage]}")
    print("=" * 50)
    
    stage_results = []
    
    # Оцениваем каждый тикер на текущем этапе
    for ticker in tickers:
        if ticker in data:
            print(f"\\n  Обрабатываем {ticker}...")
            result = evaluate_model_for_ticker_llm(data[ticker], ticker, stage, predictor)
            if result is not None:
                all_results.append(result)
                stage_results.append(result)
            
            # Небольшая пауза между запросами к API
            time.sleep(2)
    
    # Вычисляем средние метрики по этапу
    if stage_results:
        avg_rmse = np.mean([r['rmse'] for r in stage_results])
        avg_mape = np.mean([r['mape'] for r in stage_results])
        avg_da = np.mean([r['da'] for r in stage_results if not np.isnan(r['da'])])
        avg_features = np.mean([r['feature_count'] for r in stage_results])
        
        stage_summary = {
            'stage': stage,
            'stage_name': stage_names[stage],
            'avg_rmse': avg_rmse,
            'avg_mape': avg_mape,
            'avg_da': avg_da,
            'avg_features': avg_features,
            'ticker_count': len(stage_results)
        }
        
        stage_summaries.append(stage_summary)
        
        print(f"\\n📈 Средние результаты этапа {stage}:")
        print(f"   RMSE: {avg_rmse:.4f}")
        print(f"   MAPE: {avg_mape:.2f}%")
        print(f"   DA: {avg_da:.3f}")
        print(f"   Признаков: {avg_features:.1f}")
        print(f"   Успешных тикеров: {len(stage_results)}/{len(tickers)}")
    else:
        print(f"❌ Нет успешных результатов для этапа {stage}")
    
    print("\\n" + "="*50 + "\\n")

print(f"✅ Анализ завершен! Собрано {len(all_results)} результатов из {len(stage_summaries)} этапов")


In [ ]:
# Определяем названия этапов
stage_names = {
    1: "Базовая модель (close)",
    2: "+ Аномалии", 
    3: "+ Новости",
    4: "+ Свечи",
    5: "+ Технические индикаторы",
    6: "+ PCA компоненты",
    7: "+ TSFresh признаки", 
    8: "+ Картинка",
}

# Контейнер для результатов
all_results = []
stage_summaries = []

print("🚀 Начинаем прогрессивный анализ влияния признаков\n")

## TODO: adapt to LLM

# Проходим по всем этапам
for stage in range(1, 9):
    print(f"📊 ЭТАП {stage}: {stage_names[stage]}")
    print("=" * 50)
    
    stage_results = []
    
    # Оцениваем каждый тикер на текущем этапе
    for ticker in tickers:
        if ticker in data:
            result = evaluate_model_for_ticker_llm(data[ticker], ticker, stage)
            if result is not None:
                all_results.append(result)
                stage_results.append(result)
    
    # Вычисляем средние метрики по этапу
    if stage_results:
        avg_rmse = np.mean([r['rmse'] for r in stage_results])
        avg_mape = np.mean([r['mape'] for r in stage_results])
        avg_da = np.mean([r['da'] for r in stage_results if not np.isnan(r['da'])])
        avg_features = np.mean([r['feature_count'] for r in stage_results])
        
        stage_summary = {
            'stage': stage,
            'stage_name': stage_names[stage],
            'avg_rmse': avg_rmse,
            'avg_mape': avg_mape,
            'avg_da': avg_da,
            'avg_features': avg_features,
            'ticker_count': len(stage_results)
        }
        
        stage_summaries.append(stage_summary)
        
        print(f"\n📈 Средние результаты этапа {stage}:")
        print(f"   RMSE: {avg_rmse:.4f}")
        print(f"   MAPE: {avg_mape:.2f}%")
        print(f"   DA: {avg_da:.3f}")
        print(f"   Признаков: {avg_features:.1f}")
        print(f"   Успешных тикеров: {len(stage_results)}/{len(tickers)}")
    else:
        print(f"❌ Нет успешных результатов для этапа {stage}")
    
    print("\\n" + "="*50 + "\\n")

print(f"✅ Анализ завершен! Собрано {len(all_results)} результатов из {len(stage_summaries)} этапов")

🚀 Начинаем прогрессивный анализ влияния признаков

📊 ЭТАП 1: Базовая модель (close)


## Итоговая таблица результатов


In [ ]:
# Создаем итоговую таблицу
results_df = pd.DataFrame(stage_summaries)

if not results_df.empty:
    print("📊 ИТОГОВАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
    print("=" * 80)
    
    # Форматируем таблицу для красивого отображения
    display_df = results_df.copy()
    display_df['RMSE'] = display_df['avg_rmse'].apply(lambda x: f"{x:.4f}")
    display_df['MAPE (%)'] = display_df['avg_mape'].apply(lambda x: f"{x:.2f}")
    display_df['DA'] = display_df['avg_da'].apply(lambda x: f"{x:.3f}")
    display_df['Признаков'] = display_df['avg_features'].apply(lambda x: f"{x:.0f}")
    display_df['Тикеров'] = display_df['ticker_count'].apply(lambda x: f"{x}")
    
    # Выбираем колонки для отображения
    final_table = display_df[['stage', 'stage_name', 'RMSE', 'MAPE (%)', 'DA', 'Признаков', 'Тикеров']].copy()
    final_table.columns = ['Этап', 'Описание', 'RMSE', 'MAPE (%)', 'DA', 'Признаков', 'Тикеров']
    
    print(final_table.to_string(index=False))
    
    # Сохраняем таблицу
    results_df.to_csv(f"{OUTPUT_PATH}progressive_analysis_summary.csv", index=False)
    final_table.to_csv(f"{OUTPUT_PATH}progressive_analysis_formatted.csv", index=False)
    
    print(f"\\n💾 Результаты сохранены в {OUTPUT_PATH}")
    
    # Показываем изменения метрик относительно базовой модели
    if len(results_df) > 1:
        base_rmse = results_df.iloc[0]['avg_rmse']
        base_mape = results_df.iloc[0]['avg_mape'] 
        base_da = results_df.iloc[0]['avg_da']
        
        print("\\n📈 ИЗМЕНЕНИЯ ОТНОСИТЕЛЬНО БАЗОВОЙ МОДЕЛИ:")
        print("=" * 60)
        
        for i, row in results_df.iterrows():
            if i == 0:
                continue  # Пропускаем базовую модель
                
            rmse_change = ((row['avg_rmse'] - base_rmse) / base_rmse) * 100
            mape_change = ((row['avg_mape'] - base_mape) / base_mape) * 100
            da_change = ((row['avg_da'] - base_da) / base_da) * 100
            
            print(f"Этап {row['stage']} - {row['stage_name']}:")
            print(f"  RMSE: {rmse_change:+.1f}% ({'улучшение' if rmse_change < 0 else 'ухудшение'})")
            print(f"  MAPE: {mape_change:+.1f}% ({'улучшение' if mape_change < 0 else 'ухудшение'})")
            print(f"  DA: {da_change:+.1f}% ({'улучшение' if da_change > 0 else 'ухудшение'})")
            print()
            
else:
    print("❌ Нет данных для создания итоговой таблицы")

## Визуализация изменения метрик


In [ ]:
if not results_df.empty:
    # Создаем графики изменения метрик
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Прогрессивное изменение метрик качества прогнозирования', fontsize=16, fontweight='bold')
    
    # График 1: RMSE
    axes[0,0].plot(results_df['stage'], results_df['avg_rmse'], 'o-', linewidth=2, markersize=8, color='red')
    axes[0,0].set_title('RMSE (среднеквадратичная ошибка)', fontweight='bold')
    axes[0,0].set_xlabel('Этап')
    axes[0,0].set_ylabel('RMSE')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].set_xticks(results_df['stage'])
    
    # График 2: MAPE
    axes[0,1].plot(results_df['stage'], results_df['avg_mape'], 'o-', linewidth=2, markersize=8, color='orange')
    axes[0,1].set_title('MAPE (средняя абсолютная процентная ошибка)', fontweight='bold')
    axes[0,1].set_xlabel('Этап')
    axes[0,1].set_ylabel('MAPE (%)')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].set_xticks(results_df['stage'])
    
    # График 3: DA
    axes[1,0].plot(results_df['stage'], results_df['avg_da'], 'o-', linewidth=2, markersize=8, color='green')
    axes[1,0].set_title('DA (точность направления)', fontweight='bold')
    axes[1,0].set_xlabel('Этап')
    axes[1,0].set_ylabel('DA')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].set_xticks(results_df['stage'])
    axes[1,0].set_ylim(0, 1)
    
    # График 4: Количество признаков
    axes[1,1].plot(results_df['stage'], results_df['avg_features'], 'o-', linewidth=2, markersize=8, color='purple')
    axes[1,1].set_title('Количество признаков', fontweight='bold')
    axes[1,1].set_xlabel('Этап')
    axes[1,1].set_ylabel('Количество признаков')
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].set_xticks(results_df['stage'])
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_PATH}metrics_progression.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Создаем heatmap сравнения метрик
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    # Подготавливаем данные для heatmap
    heatmap_data = results_df[['stage', 'avg_rmse', 'avg_mape', 'avg_da']].copy()
    
    # Нормализуем данные для лучшей визуализации (min-max scaling)
    for col in ['avg_rmse', 'avg_mape', 'avg_da']:
        min_val = heatmap_data[col].min()
        max_val = heatmap_data[col].max()
        if max_val > min_val:
            heatmap_data[col] = (heatmap_data[col] - min_val) / (max_val - min_val)
    
    # Для RMSE и MAPE - инвертируем (меньше = лучше)
    heatmap_data['avg_rmse'] = 1 - heatmap_data['avg_rmse']
    heatmap_data['avg_mape'] = 1 - heatmap_data['avg_mape']
    
    # Создаем heatmap
    heatmap_matrix = heatmap_data[['avg_rmse', 'avg_mape', 'avg_da']].T
    heatmap_matrix.columns = [f"Этап {i}" for i in results_df['stage']]
    heatmap_matrix.index = ['RMSE (норм.)', 'MAPE (норм.)', 'DA']
    
    sns.heatmap(heatmap_matrix, annot=True, cmap='RdYlGn', center=0.5, 
                cbar_kws={'label': 'Качество (нормализовано)'}, ax=ax)
    ax.set_title('Тепловая карта качества моделей по этапам\\n(зеленый = лучше, красный = хуже)', 
                 fontweight='bold', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_PATH}quality_heatmap.png", dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("❌ Нет данных для визуализации")

## Выводы и рекомендации


In [ ]:
print("🎯 РЕЗЮМЕ ПРОГРЕССИВНОГО АНАЛИЗА")
print("=" * 50)

if results_df.empty:
    print("❌ Нет результатов для анализа")
else:
    print(f"✅ Проанализировано {len(results_df)} этапов для {len(tickers)} тикеров")
    print(f"📊 Собрано {len(all_results)} успешных результатов")
    
    # Найдем лучший этап по каждой метрике
    best_rmse_stage = results_df.loc[results_df['avg_rmse'].idxmin()]
    best_mape_stage = results_df.loc[results_df['avg_mape'].idxmin()]
    best_da_stage = results_df.loc[results_df['avg_da'].idxmax()]
    
    print("\\n🏆 ЛУЧШИЕ РЕЗУЛЬТАТЫ:")
    print(f"   Лучший RMSE: Этап {best_rmse_stage['stage']} ({best_rmse_stage['stage_name']}) - {best_rmse_stage['avg_rmse']:.4f}")
    print(f"   Лучший MAPE: Этап {best_mape_stage['stage']} ({best_mape_stage['stage_name']}) - {best_mape_stage['avg_mape']:.2f}%")
    print(f"   Лучший DA: Этап {best_da_stage['stage']} ({best_da_stage['stage_name']}) - {best_da_stage['avg_da']:.3f}")
    
    # Анализ тренда качества
    base_metrics = results_df.iloc[0]
    final_metrics = results_df.iloc[-1]
    
    rmse_change = ((final_metrics['avg_rmse'] - base_metrics['avg_rmse']) / base_metrics['avg_rmse']) * 100
    mape_change = ((final_metrics['avg_mape'] - base_metrics['avg_mape']) / base_metrics['avg_mape']) * 100
    da_change = ((final_metrics['avg_da'] - base_metrics['avg_da']) / base_metrics['avg_da']) * 100
    
    print("\\n📈 ОБЩЕЕ УЛУЧШЕНИЕ (финальный этап vs базовая модель):")
    print(f"   RMSE: {rmse_change:+.1f}% ({'✅ улучшение' if rmse_change < 0 else '❌ ухудшение'})")
    print(f"   MAPE: {mape_change:+.1f}% ({'✅ улучшение' if mape_change < 0 else '❌ ухудшение'})")
    print(f"   DA: {da_change:+.1f}% ({'✅ улучшение' if da_change > 0 else '❌ ухудшение'})")
    
    # Рекомендации
    print("\\n💡 РЕКОМЕНДАЦИИ:")
    
    # Определяем наиболее эффективные этапы
    rmse_improvements = []
    mape_improvements = []
    da_improvements = []
    
    for i in range(1, len(results_df)):
        prev_metrics = results_df.iloc[i-1]
        curr_metrics = results_df.iloc[i]
        
        rmse_change = ((curr_metrics['avg_rmse'] - prev_metrics['avg_rmse']) / prev_metrics['avg_rmse']) * 100
        mape_change = ((curr_metrics['avg_mape'] - prev_metrics['avg_mape']) / prev_metrics['avg_mape']) * 100
        da_change = ((curr_metrics['avg_da'] - prev_metrics['avg_da']) / prev_metrics['avg_da']) * 100
        
        rmse_improvements.append((curr_metrics['stage'], rmse_change))
        mape_improvements.append((curr_metrics['stage'], mape_change))
        da_improvements.append((curr_metrics['stage'], da_change))
    
    # Находим этапы с наибольшими улучшениями
    best_rmse_improvement = min(rmse_improvements, key=lambda x: x[1])
    best_mape_improvement = min(mape_improvements, key=lambda x: x[1])
    best_da_improvement = max(da_improvements, key=lambda x: x[1])
    
    print(f"   1. Наибольшее улучшение RMSE дал этап {best_rmse_improvement[0]} ({best_rmse_improvement[1]:+.1f}%)")
    print(f"   2. Наибольшее улучшение MAPE дал этап {best_mape_improvement[0]} ({best_mape_improvement[1]:+.1f}%)")
    print(f"   3. Наибольшее улучшение DA дал этап {best_da_improvement[0]} ({best_da_improvement[1]:+.1f}%)")
    
    # Анализ эффективности по соотношению качества к сложности
    print("\\n⚖️ АНАЛИЗ ЭФФЕКТИВНОСТИ (качество vs сложность):")
    for _, row in results_df.iterrows():
        efficiency_score = (1 - row['avg_rmse']/base_metrics['avg_rmse']) / (row['avg_features']/base_metrics['avg_features'])
        print(f"   Этап {row['stage']}: Эффективность = {efficiency_score:.3f} (качество/сложность)")

print("\\n🎉 ПРОГРЕССИВНЫЙ АНАЛИЗ ЗАВЕРШЕН!")
print(f"📁 Все результаты сохранены в папке: {OUTPUT_PATH}")
print("\\nФайлы результатов:")
print("   - progressive_analysis_summary.csv - сводка по этапам")
print("   - progressive_analysis_formatted.csv - форматированная таблица")
print("   - feature_importance_summary.csv - важность признаков")
print("   - metrics_progression.png - графики изменения метрик")
print("   - quality_heatmap.png - тепловая карта качества")
print("   - feature_importance_stage_N.png - важность признаков для последнего этапа")